# Settings and utility functions

This module has all of the standard pieces of code we need for the tutrials so that we don't have to repeat them for every lesson.

## Set path to CADET bin folder

The first step is to import CADET and tell the system where cadet-cli can be found.

<div class="alert alert-info">

**Note:** 

Please specify the location of the `cadet-cli` executable. It is located in the bin folder where CADET was installed.

</div>

In [2]:
import shutil
import os
import platform
from pathlib import Path
from cadet import Cadet

# Either ensure CADET is on your PATH (e.g. by installing via conda)
# OR 
# provide the path to the CADET installation
# E.g.
#     windows: C:\Users\<username>\cadet
#     linux: ~/cadet/bin
# would be set by:

install_path = 'C:/Users/hoang/miniconda3/envs/cadet-env/'

executable = 'cadet-cli.exe'
if install_path is None:
    try:
        if platform.system() == 'Windows':
            executable += '.exe'
        executable_path = Path(shutil.which(executable))
    except TypeError:
        raise FileNotFoundError(
            "CADET could not be found. Please set an install path"
        )
    install_path = executable_path.parent.parent

install_path = Path(install_path)
cadet_bin_path = install_path / "bin" / executable

if cadet_bin_path.exists():
    Cadet.cadet_path = cadet_bin_path
else:
    raise FileNotFoundError(
        "CADET could not be found. Please check the path"
    )

cadet_lib_path = install_path / "lib"
try:
    if cadet_lib_path.as_posix() not in os.environ['LD_LIBRARY_PATH']:
        os.environ['LD_LIBRARY_PATH'] = \
            cadet_lib_path.as_posix() \
            + os.pathsep \
            + os.environ['LD_LIBRARY_PATH']
except KeyError:
    os.environ['LD_LIBRARY_PATH'] = cadet_lib_path.as_posix()

lwe_executable = 'createLWE'
if platform.system() == 'Windows':
    lwe_executable += '.exe'
lwe_path = install_path / "bin" / lwe_executable

## Standard imports

Here, some other standard libraries are imported.

In [ ]:
import os

from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.display import Image

# python numeric library
import numpy as np

# scientific library for python
import scipy

# pandas is python library for data analysis
import pandas

# addict is a library that makes it easier to create nested dictionaries
from addict import Dict

# json is a standard text based format and it used in CADETMatch for the configuration file
import json

# python plotting library
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='svg'
%matplotlib inline

# jupyter widget support
from ipywidgets import interact, interactive
import ipywidgets as widgets

# Temporary files for simulation objects
import tempfile
tempfile.tempdir = os.path.join(Path.home())

import subprocess

## Template for CADET simulations

This function defines some default values that can be used for all simulations.

In [ ]:
def get_cadet_template(n_units=3, split_components_data=False):
    cadet_template = Cadet()
    
    cadet_template.root.input.model.nunits = n_units
    
    # Store solution
    cadet_template.root.input['return'].split_components_data = split_components_data
    cadet_template.root.input['return'].split_ports_data = 0
    cadet_template.root.input['return'].unit_000.write_solution_inlet = 1
    cadet_template.root.input['return'].unit_000.write_solution_outlet = 1
    cadet_template.root.input['return'].unit_000.write_solution_bulk = 1
    cadet_template.root.input['return'].unit_000.write_solution_particle = 1
    cadet_template.root.input['return'].unit_000.write_solution_solid = 1
    cadet_template.root.input['return'].unit_000.write_solution_flux = 1
    cadet_template.root.input['return'].unit_000.write_solution_volume = 1
    cadet_template.root.input['return'].unit_000.write_coordinates = 1
    cadet_template.root.input['return'].unit_000.write_sens_outlet = 1
    
    for unit in range(n_units):
        cadet_template.root.input['return']['unit_{0:03d}'.format(unit)] = cadet_template.root.input['return'].unit_000
        
    # Tolerances for the time integrator
    cadet_template.root.input.solver.time_integrator.abstol = 1e-6
    cadet_template.root.input.solver.time_integrator.algtol = 1e-10
    cadet_template.root.input.solver.time_integrator.reltol = 1e-6
    cadet_template.root.input.solver.time_integrator.init_step_size = 1e-6
    cadet_template.root.input.solver.time_integrator.max_steps = 1000000
    
    # Solver settings
    cadet_template.root.input.model.solver.gs_type = 1
    cadet_template.root.input.model.solver.max_krylov = 0
    cadet_template.root.input.model.solver.max_restarts = 10
    cadet_template.root.input.model.solver.schur_safety = 1e-8

    # Run the simulation on single thread
    cadet_template.root.input.solver.nthreads = 1
    
    return cadet_template


def set_discretization(model, n_bound=None, n_col=20, n_par_types=1):
    columns = {'GENERAL_RATE_MODEL', 'LUMPED_RATE_MODEL_WITH_PORES', 'LUMPED_RATE_MODEL_WITHOUT_PORES'}
    
    
    for unit_name, unit in model.root.input.model.items():
        if 'unit_' in unit_name and unit.unit_type in columns:
            unit.discretization.ncol = n_col
            unit.discretization.npar = 5
            unit.discretization.npartype = n_par_types
            
            if n_bound is None:
                n_bound = unit.ncomp*[0]
            unit.discretization.nbound = n_bound
            
            unit.discretization.par_disc_type = 'EQUIDISTANT_PAR'
            unit.discretization.use_analytic_jacobian = 1
            unit.discretization.reconstruction = 'WENO'
            unit.discretization.gs_type = 1
            unit.discretization.max_krylov = 0
            unit.discretization.max_restarts = 10
            unit.discretization.schur_safety = 1.0e-8

            unit.discretization.weno.boundary_model = 0
            unit.discretization.weno.weno_eps = 1e-10
            unit.discretization.weno.weno_order = 3

This function creates a simple model for a dextran pulse.

In [ ]:
def save_to_csv(time, c, file_name):
    combined_data = np.column_stack(time, c)
    
    np.save_to_csv(file_name, combined_data, delimiter=',')

## Function for running CADET

Wrapper for calling the `Cadet.run()` function with some additional functionality.

In [ ]:
def run_simulation(cadet, file_name=None):
    if file_name is None:
        f = next(tempfile._get_candidate_names())
        cadet.filename = os.path.join(tempfile.tempdir, f + '.h5')
    else:
        cadet.filename = file_name
    # save the simulation
    cadet.save()

    # run the simulation and load results
    data = cadet.run()
    cadet.load()
    
    # Remove files 
    if file_name is None:
        os.remove(os.path.join(tempfile.tempdir, f + '.h5'))

    # Raise error if simulation fails
    if data.returncode == 0:
        print("Simulation completed successfully")
    else:
        print(data)
        raise Exception("Simulation failed")